--- Day 7: Laboratories ---

You thank the cephalopods for the help and exit the trash compactor, finding yourself in the familiar halls of a North Pole research wing.

Based on the large sign that says "teleporter hub", they seem to be researching teleportation; you can't help but try it for yourself and step onto the large yellow teleporter pad.

Suddenly, you find yourself in an unfamiliar room! The room has no doors; the only way out is the teleporter. Unfortunately, the teleporter seems to be leaking magic smoke.

Since this is a teleporter lab, there are lots of spare parts, manuals, and diagnostic equipment lying around. After connecting one of the diagnostic tools, it helpfully displays error code 0H-N0, which apparently means that there's an issue with one of the tachyon manifolds.

You quickly locate a diagram of the tachyon manifold (your puzzle input). A tachyon beam enters the manifold at the location marked S; tachyon beams always move downward. Tachyon beams pass freely through empty space (.). However, if a tachyon beam encounters a splitter (^), the beam is stopped; instead, a new tachyon beam continues from the immediate left and from the immediate right of the splitter.

For example:

.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............

In this example, the incoming tachyon beam (|) extends downward from S until it reaches the first splitter:

.......S.......
.......|.......
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............

At that point, the original beam stops, and two new beams are emitted from the splitter:

.......S.......
.......|.......
......|^|......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............

Those beams continue downward until they reach more splitters:

.......S.......
.......|.......
......|^|......
......|.|......
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............

At this point, the two splitters create a total of only three tachyon beams, since they are both dumping tachyons into the same place between them:

.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............

This process continues until all of the tachyon beams reach a splitter or exit the manifold:

.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|

To repair the teleporter, you first need to understand the beam-splitting properties of the tachyon manifold. In this example, a tachyon beam is split a total of 21 times.

Analyze your manifold diagram. How many times will the beam be split?


In [1]:
test_path = "../test/test07_1.txt"
file_path = "day07.txt"


In [2]:
def get_grid(filepath: str = test_path) -> dict:
    ''''
    Gets the grid for the beams from the input file
    '''

    with open(filepath, "r") as file:
        lines = file.readlines()

    grid = dict()

    for i, line in enumerate(lines):
        for j, col in enumerate(line.strip()):
            grid[(i,j)] = col
    
    return grid

In [3]:
grid = get_grid()

In [4]:
def convert_to_print_grid(grid: dict) -> list:

    '''
    converts the grid to a string format so it can be printed nicely
    '''

    height, width = sorted(list(grid.keys()))[-1]
    # create a nice visual representation to print
    print_grid = []
    for h in range(height):
        row = ""
        
        for w in range(width+h):
            row += grid.get((h,w), "")
            
        print_grid.append(row)
        
    return print_grid
    


In [5]:
def process_beams(grid: dict, print_grid: bool = False) -> int:

    
    height, width = sorted(list(grid.keys()))[-1]
    start = [k for k, v in grid.items() if v == "S"]
    new_grid = {k: v for k, v in grid.items()}
    total = 0
    
    #  trace the beam path
    for h in range(height):
        for w in range(width):
            pos = h, w
            if new_grid[pos] == "S":
                new_grid[(h+1, w)] = "|"
            elif grid[pos] == "^":
                above, left, right = (h-1, w), (h, w-1), (h, w+1)
                below_left, below_right = (h+1, w-1), (h+1, w+1)
                if new_grid.get(above, ".") == "|":
                    split_left, split_right = False, False
                    
                    # if we have "-" we are outside our grid
                    if grid.get(left, "-") not in ["-", "^"]:
                        new_grid[left] = "|"
                        if grid.get(below_left, ".") != "^":
                            new_grid[below_left] = "|"
                            split_left = True
                    if grid.get(right, "-") not in ["-", "^"]:
                        new_grid[right] = "|"
                        if grid.get(below_right, ".") != "^":
                            new_grid[below_right] = "|"
                            split_right = True
                    total += split_left and split_right
            if new_grid[pos] == "|":
                if new_grid.get((h+1, w), ".") == ".":
                    new_grid[(h+1, w)] = "|"

    printable_grid, printable_new_grid = convert_to_print_grid(grid), convert_to_print_grid(new_grid)

    if print_grid:
        print("Manifold without the beams")
        [print(row) for row in printable_grid]
        print("------------------------------------------------------------------")
        print("Manifold with the tachyon beams")
        [print(row) for row in printable_new_grid]
        
    return total, new_grid

In [6]:
def day07a(filepath: str = test_path, print_grid: bool = False) -> int:
    '''
    Calculates the number of times is split in the manifold and returns it as an int.
    '''
    
    grid = get_grid(filepath)
    result, _ = process_beams(grid, print_grid)

    print("Day 07 part a:")
    print(f"The tachyon beam is slit {result} times in the manifold")
    return result
    

In [7]:
def test_day07a():
    assert day07a() == 21

In [8]:
test_day07a()

Day 07 part a:
The tachyon beam is slit 21 times in the manifold


In [9]:
day07a(file_path)

Day 07 part a:
The tachyon beam is slit 1633 times in the manifold


1633

--- Part Two ---

With your analysis of the manifold complete, you begin fixing the teleporter. However, as you open the side of the teleporter to replace the broken manifold, you are surprised to discover that it isn't a classical tachyon manifold - it's a quantum tachyon manifold.

With a quantum tachyon manifold, only a single tachyon particle is sent through the manifold. A tachyon particle takes both the left and right path of each splitter encountered.

Since this is impossible, the manual recommends the many-worlds interpretation of quantum tachyon splitting: each time a particle reaches a splitter, it's actually time itself which splits. In one timeline, the particle went left, and in the other timeline, the particle went right.

To fix the manifold, what you really need to know is the number of timelines active after a single particle completes all of its possible journeys through the manifold.

In the above example, there are many timelines. For instance, there's the timeline where the particle always went left:

.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
...|^.^...^....
...|...........
..|^.^...^.^...
..|............
.|^...^.....^..
.|.............
|^.^.^.^.^...^.
|..............

Or, there's the timeline where the particle alternated going left and right at each splitter:

.......S.......
.......|.......
......|^.......
......|........
......^|^......
.......|.......
.....^|^.^.....
......|........
....^.^|..^....
.......|.......
...^.^.|.^.^...
.......|.......
..^...^|....^..
.......|.......
.^.^.^|^.^...^.
......|........

Or, there's the timeline where the particle ends up at the same point as the alternating timeline, but takes a totally different path to get there:

.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
....^|^...^....
.....|.........
...^.^|..^.^...
......|........
..^..|^.....^..
.....|.........
.^.^.^|^.^...^.
......|........

In this example, in total, the particle ends up on 40 different timelines.

Apply the many-worlds interpretation of quantum tachyon splitting to your manifold diagram. In total, how many different timelines would a single tachyon particle end up on?


In [10]:
def find_paths(pos: tuple, grid: dict) -> int:
    '''
    Calculates the number of paths from the given position available and returns the number of paths as an int.
    '''
    #print(pos)
    x, y = pos
    new_pos_left, new_pos, new_pos_right = (x+1, y-1), (x+1, y), (x+1, y+1)
    total = 1
    below = grid.get(new_pos, ".") 
    if below == ".":
        return 1
    elif below == "^":
        return find_paths(new_pos_left, grid) + find_paths(new_pos_right, grid)
    elif below == "|":
        return find_paths(new_pos, grid)

The functionality works as intended but we need memoization to actually make the code useful (because of the exploding number of redundant function calls. To this end we need @lru_cache for the memoization. This 

In [11]:
import functools
from frozendict import frozendict

def freezeargs(func):
    """Convert a mutable dictionary into immutable.
    Useful to be compatible with cache
    """

    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        args = (frozendict(arg) if isinstance(arg, dict) else arg for arg in args)
        kwargs = {k: frozendict(v) if isinstance(v, dict) else v for k, v in kwargs.items()}
        return func(*args, **kwargs)
    return wrapped

In [12]:
from functools import lru_cache

@freezeargs
@lru_cache(maxsize=None) 
def find_paths(pos: tuple, grid: dict) -> int:
    '''
    Calculates the number of paths from the given position available and returns the number of paths as an int.
    '''
    x, y = pos
    new_pos_left, new_pos, new_pos_right = (x+1, y-1), (x+1, y), (x+1, y+1)
    total = 1
    below = grid.get(new_pos, ".") 
    if below == ".":
        return 1
    elif below == "^":
        return find_paths(new_pos_left, grid) + find_paths(new_pos_right, grid)
    elif below == "|":
        return find_paths(new_pos, grid)
    

In [13]:
def day07b(filepath: str = test_path, print_grid: bool = False) -> int:

    '''
    Calculates the number of possible paths that the tachyon beam can take and returns it as an int.
    '''

    from functools import lru_cache

    grid = get_grid(filepath)
    _, tachyon_grid = process_beams(grid, print_grid)
    starting_point = [k for k,v in tachyon_grid.items() if k[0] == 1 and v == "|"][0]
    number_of_paths = find_paths(starting_point, tachyon_grid)

    print("Day 07 part b:")
    print(f"There are {number_of_paths} possible paths available to the tachyon beam.")

    return number_of_paths
    

In [14]:
def test_day07b():
    assert day07b() == 40

In [15]:
test_day07b()

Day 07 part b:
There are 40 possible paths available to the tachyon beam.


In [16]:
day07b(file_path)

Day 07 part b:
There are 34339203133559 possible paths available to the tachyon beam.


34339203133559